### BlueBike AI: Using AI to Predict best time to make BlueBike Trip
$\newline$
Final project $\newline$
Professor Snyder's CS4100 Artificial Intelligence class
$\newline$
Group members: Ariel Mahler, Omri Leshem, Marco Tortolani
GitHub Repo: https://github.com/Mtortolani/bluebike-ai

### Motivation
For this project, our group wanted to address a problem that has a tangible effect on our lives. Transportation in Boston is always a challenge for us, especially for group member Omri, who often has challenges finding a good time to grab a BlueBike. BlueBike is a subscription based service that allows you to pick up an available BlueBike from a station at any time, and return it to a station once your trip is over. The challenge is that sometimes, when going to a station, you can find that it is out of bikes, or at the end of a trip, that the docking station we were heading to doesn't have any available parking spots. Both of these are challenges that require us to look for alternative stations, which takes time and effort. We hope that through this project we can create a tool to help reduce the overall time spent finding stations to undock/dock BlueBikes, and eliminating guesswork when planning a trip.

### System Design Overview
The challenge of this project is to predict the best time to take a trip in a BlueBike to have the best odds at an available bike at starting station and an available bike at end station. For us to accurately predict this, we devised 3 core components: 
1. Feed in historical BlueBike trip data and process that into something usable
2. Develop a machine learning model that takes in historical station data to predict how many bikes would be at a station at a given time
3. Design a controller that would query the model and apply a heuristic to deduce the best time to take a BlueBike trip.

This framework allowed us to break apart the task into three individual Python classes, which helped us segment the work. Although we started off with Omri working on DataCleaning.py (the data), Marco working on BikeCountPredictor.py (the model), and Ari working on BikeCountController (the heurostic), it very quickly turned into a joint effort to figure out how to find more data to improve the model, or making the heuristic more sensible to a rider's real trip.

### Data Cleaning
This section of the program extracts the usable/necessary data from the BlueBike historical datasets (ranging from April 2022 to March 2023), and processes it for use by the model.

In [8]:
import pandas as pd
from pandas import to_datetime
import seaborn as sns
import matplotlib.pyplot as plti

#balancing first
def rebalance(year_month):
    #NOTE: Jupyter notebook does not always work with relative path
    # if error, replace with absolute path
    df = pd.read_csv("./src/data/%s-bluebikes-tripdata.csv" % year_month,
            usecols=['starttime','start station id',
                    'stoptime','end station id', 'bikeid'],
            parse_dates=['starttime','stoptime'])

    dfbike=df.sort_values(by=['bikeid','starttime'])
    dfbike.head(10)

    offset = pd.DataFrame({'starttime': pd.to_datetime('2010-09-01'),
    'start station id':0,'stoptime': pd.to_datetime('2010-09-01'),
    'end station id':0,'bikeid':0},index=[0])

    dfbike1 = pd.concat([offset,dfbike]).reset_index(drop=True)
    dfbike2 = pd.concat([dfbike,offset]).reset_index(drop=True)

    dfbike=pd.concat ([dfbike1[['bikeid','stoptime','end station id']]\
                ,dfbike2[['bikeid','starttime','start station id']] ],\
                axis=1 )
    dfbike.head()

    dfbike.columns=['bikeid1','starttime','start station id',\
                    'bikeid2','stoptime','end station id']
    dfrebal = dfbike[['starttime','start station id',\
                    'stoptime','end station id']].\
            loc[(dfbike.bikeid1==dfbike.bikeid2) & \
            (dfbike['start station id'] != dfbike['end station id']) ]
    dfrebal.reset_index(drop=True, inplace=True)
    dfrebal
    dfrebal.to_parquet('./src/data/%s-bluebike-reblance.parquet' % year_month)
    print("Rebalancing done for %s!" % year_month)

#create csv for number of bikes at particular station, using the rebalanced parquet file
def bikeAvail(station, year_month):
    
    df = pd.read_csv('./src/data/%s-bluebikes-tripdata.csv' % year_month,
            usecols=['starttime','start station id',
                    'stoptime','end station id'],
            parse_dates=['starttime','stoptime'])
    dfrebal = pd.read_parquet ('./src/data/%s-bluebike-reblance.parquet' % year_month)
    df = pd.concat([df,dfrebal])
    df.reset_index(drop=True, inplace=True)

    dfs=df[['starttime','start station id']].assign(act=-1)
    dfe=df[['stoptime','end station id']].assign(act=1)
    dfs.columns=['docktime','stationid','act']
    dfe.columns=['docktime','stationid','act']
    dfse=pd.concat([dfs,dfe])


    dfse.sort_values(by=['docktime'], inplace=True) 
    dfse.reset_index(drop=True, inplace=True) 
    dfse.head(100)


    dfstations = pd.read_csv("./src/data/%s-bluebikes-tripdata.csv" % year_month,
    usecols=['start station id','start station name']).drop_duplicates()                
    dfstations.columns=['stationid','station name']
    dfstations.set_index('stationid',drop=True, inplace=True)

    station_id = dfstations.loc[dfstations['station name']\
     == station].index[0]
    dfStation = dfse.loc[(dfse.stationid==station_id) ]
    dfStation.reset_index(drop=True, inplace=True)

    dfStation = dfStation.assign(num_bikes = dfStation.act.cumsum())
    dfStation.at[0, 'act'] += abs(dfStation.act.cumsum().min())
    dfStation = dfStation.assign(num_bikes = dfStation.act.cumsum())
    dfStation['minutes'] = 0
    dfStation = dfStation.assign(minutes =  dfStation['docktime'].dt.hour * 60 + dfStation['docktime'].dt.minute + round(dfStation['docktime'].dt.second / 60, 3))
    dfStation = dfStation.drop('act', axis = 1)
    print("Dock data generated for %s" % station)

    attachClimateData(dfStation)


#clean climate data csv to include only daily average temp in boston over the year we're looking at. Only needs to be run once.
def cleanClimateData():
        dfClimate = pd.read_csv('./src/data/2022-2023-climate-data.csv',
                usecols=['DATE','DailyAverageDryBulbTemperature'],
                parse_dates=['DATE'])
        dfClimate = dfClimate.dropna()
        dfClimate.reset_index(inplace=True)
        dfClimate.to_csv('./src/data/2022-2023-climate-data-processed.csv')
        print(dfClimate.head(10))
        print(dfClimate.index)

#merge the climate data and the dock data so that every day has an average temperature.
def attachClimateData(df2):
        df1 = pd.read_csv('./src/data/2022-2023-climate-data-processed.csv',
                usecols=['DATE','DailyAverageDryBulbTemperature'],
                parse_dates=['DATE'])
        df1.rename(columns = {'DATE': 'docktime'}, inplace = True)
        df1.docktime = df1.docktime.apply(lambda x: x.date())
        df2.docktime = df2.docktime.apply(lambda x: x.date())

        merged_df = pd.merge(df2, df1, how='left', on= 'docktime')
        merged_df.rename(columns = {'docktime': 'day'}, inplace = True)
        merged_df.rename(columns = {'DailyAverageDryBulbTemperature': 'average_temp'}, inplace = True)
        merged_df.to_csv('./data/2022-2023-dock-data-updated.csv', mode='a', header=False)
        print("Average temp attached to dock data!")

def addMoreParams():
       dfA = pd.read_csv('./src/data/2022-2023-dock-data-updated.csv',
                         usecols=['day', 'stationid', 'num_bikes', 'minutes', 'average_temp'],
                        parse_dates=['day'])
       #rename day column to date
       dfA.rename(columns = {'day': 'date'}, inplace = True)
#        #add day of week
       dfA['day'] = dfA.date.apply(lambda x: x.date().day)
       dfA['month'] = dfA.date.apply(lambda x: x.date().month)


       #add weekday (as in Monday, Tuesday, etc.)
       dfA['day_of_week'] = dfA.date.apply(lambda x: x.weekday())

       dfA.to_csv('./data/2022-2023-dock-data-updated-fo-real.csv', index=False)

def doTheThing():
        array_of_year_months = ["202302", "202301", "202212", "202211", "202210", "202209", "202208", "202207", "202206", "202205", "202204"]
        start_station = "Boylston St at Jersey St"
        end_station = "Ruggles T Stop - Columbus Ave at Melnea Cass Blvd"
        for year_month in array_of_year_months:
                rebalance(year_month)
                bikeAvail(start_station, year_month)
                bikeAvail(end_station, year_month)

# addMoreParams()

In [9]:
dataFilename = "./src/data/2022-2023-dock-data-updated-fo-real.csv"
df = pd.read_csv(dataFilename)
df

,date,stationid,num_bikes,minutes,average_temp,day,month,day_of_week
0,2023-03-01,342,13,77.933,38.0,1,3,2
1,2023-03-01,342,14,105.867,38.0,1,3,2
2,2023-03-01,342,15,106.333,38.0,1,3,2
3,2023-03-01,342,14,491.317,38.0,1,3,2
4,2023-03-01,342,13,492.850,38.0,1,3,2
...,...,...,...,...,...,...,...,...
112565,2022-04-30,12,25,1413.533,48.0,30,4,5
112566,2022-04-30,12,24,1423.400,48.0,30,4,5
112567,2022-04-30,12,23,1423.717,48.0,30,4,5
112568,2022-05-01,12,24,13.550,53.0,1,5,6


### Model Creation & Testing
Although the task of predicting optimal bike depature time seemed like a daunting challenge, it was made much more clear when we divided the model into seperate cleaning, predicting, and heuristic measurement. As the data cleaning took place and we were afforded more parameters as inputs, we were able to improve our models. Our first model was a simple linear regression which had a large error rate and a an r2 value of less than 0.1. With only the minutes in the day and the temperature as the inpui=t variable, the model had almost no ability to properly predict the amount of bikes in a station at a given time. As we provided more variables such as day of the week and month, we were able to implement more complex models such as a knn regressor and a decision tree regressor. Being familiar with complex models we knew we had to hyperparameter tune (particularly for decision trees as they tend to strongly overfit with too many layers). After hyperparameter tuning our new models with the new bike station historical data, we were able to achieve our best running model which is a knn regressor with a MSE of ~8 and a r2 value of ~0.62. Although this doesn't meet the bar for professional models, at least the model is improving substantially from the data we are feeding it.

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

class BikeCountPredictor:
    def __init__(self, dataFilename='./src/data/2022-2023-dock-data-updated-fo-real.csv'):
        
        #initialization functions
        self.stationData = self.loadDataByStation(dataFilename)
        self.stationModels = {} # self.stationModels[stationId: int][desired_model: str] = model
        
    def loadDataByStation(self, allStationDataFilename: str) -> dict[pd.DataFrame]:
        dfAllStations = pd.read_csv(allStationDataFilename)
        dfAllStations = dfAllStations.dropna()
        uniqueStationIds = dfAllStations['stationid'].unique()
        return {stationId : dfAllStations.loc[dfAllStations['stationid']==stationId] for stationId in uniqueStationIds}
    
    def existsStationData(self, stationId: int) -> None:
        if not self.stationData:
            raise Exception("There is no station data loaded.")
        if stationId not in self.stationData:
            raise Exception("There is no station data for this particular station.")
        
    def createModel(self, stationId: int, desired_model: str, verbose: bool = True):
        self.existsStationData(stationId)
        xCols = ["minutes", "average_temp","day","month","day_of_week"]
        dfX = self.stationData[stationId][xCols]
        dfY = self.stationData[stationId][["num_bikes"]]
        X_train, X_test, y_train, y_test = train_test_split(dfX.to_numpy(), dfY.to_numpy(), test_size=0.20, random_state=123)
        if (verbose): print(f"\nResults for {desired_model} model on station {stationId}:")
        if desired_model == "linear":
            params = {}
            model = GridSearchCV(LinearRegression() , param_grid=params, scoring='neg_mean_squared_error',cv=5)
        elif desired_model == "knn":
            params = [{'n_neighbors': [3, 5, 7, 9],
                       'weights': ['uniform', 'distance']}]
            model = GridSearchCV(KNeighborsRegressor() , param_grid=params,scoring='neg_mean_squared_error',cv=5)
        elif desired_model == "decision tree":
            params = {'max_depth': range(1,10,2), 
                          "min_samples_split": range(2,10,2), 
                          "min_samples_leaf": range(1,10,2)}
            model = GridSearchCV(DecisionTreeRegressor() , param_grid=params,scoring='neg_mean_squared_error',cv=5)
        else:
            raise Exception(f"Desired model type {desired_model} does not exist")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        if (verbose):
            print(f"The best parameters for {desired_model} model from gridsearch are {model.best_params_}")
            print(f"The test MSE for {desired_model} model is: ", mean_squared_error(y_test,y_pred))
            print(f"The R2 score {desired_model} model is: ", r2_score(y_test,y_pred))
        if not stationId in self.stationModels:
            self.stationModels[stationId] = {}
        self.stationModels[stationId][desired_model] = model
        return model
        

    def existsStationModel(self, stationId: int, desired_model: str):
        return stationId in self.stationModels and desired_model in self.stationModels[stationId]
    
    def predictBikeCount(self, stationId, minutes: float, temp: float, day: int, month: int, day_of_week: int, desired_model: str, verbose:bool = True) -> int:
        if not self.existsStationModel(stationId, desired_model):
            model =  self.createModel(stationId, desired_model, verbose)
        model = self.stationModels[stationId][desired_model]
        y_pred = model.predict([[minutes, temp, day, month, day_of_week]])
        if (verbose):
            print(f"\nThe {desired_model} model predicts that station {stationId} at minutes {minutes} and temp {temp} will have {y_pred[0]} bikes")
        return y_pred[0]
    
def main():
    BCP = BikeCountPredictor()
    stationId = 342
    minutes = 950 #15:50
    temp = 47
    day = 5
    month = 7
    day_of_week = 0
    desired_model = 'decision tree'
    BCP.predictBikeCount(stationId, minutes, temp, day, month, day_of_week, desired_model)
    
    stationId = 342
    minutes = 1200 
    temp = 64
    day = 15
    month = 5
    day_of_week = 0
    desired_model = 'decision tree'
    BCP.predictBikeCount(stationId, minutes, temp, day, month, day_of_week, desired_model)
    
    stationId = 12
    minutes = 950 #15:50
    temp = 47
    day = 25
    month = 9
    day_of_week = 2
    desired_model = 'knn'
    BCP.predictBikeCount(stationId, minutes, temp, day, month, day_of_week, desired_model)

if __name__ == "__main__":
    main()
    


Results for decision tree model on station 342:
The best parameters for decision tree model from gridsearch are {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 4}
The test MSE for decision tree model is:  10.159631777798728
The R2 score decision tree model is:  0.43091909123527683

The decision tree model predicts that station 342 at minutes 950 and temp 47 will have 4.443478260869565 bikes

The decision tree model predicts that station 342 at minutes 1200 and temp 64 will have 12.346456692913385 bikes

Results for knn model on station 12:
The best parameters for knn model from gridsearch are {'n_neighbors': 5, 'weights': 'distance'}
The test MSE for knn model is:  16.854413213813473
The R2 score knn model is:  0.6179610333816541

The knn model predicts that station 12 at minutes 950 and temp 47 will have [10.64193664] bikes


In [12]:
# Display results for creating and testing each model type on each station
BCP = BikeCountPredictor()
desired_model_types = ['linear', 'decision tree', 'knn']

for desired_model in desired_model_types:
    BCP.createModel(12, desired_model, verbose=True)
    BCP.createModel(342, desired_model, verbose=True)


Results for linear model on station 12:
The best parameters for linear model from gridsearch are {}
The test MSE for linear model is:  39.694464316197084
The R2 score linear model is:  0.10024561902872975

Results for linear model on station 342:
The best parameters for linear model from gridsearch are {}
The test MSE for linear model is:  17.348386236604092
The R2 score linear model is:  0.028248698274460304

Results for decision tree model on station 12:
The best parameters for decision tree model from gridsearch are {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 4}
The test MSE for decision tree model is:  18.671852961219738
The R2 score decision tree model is:  0.5767651285357269

Results for decision tree model on station 342:
The best parameters for decision tree model from gridsearch are {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 6}
The test MSE for decision tree model is:  10.16411908228786
The R2 score decision tree model is:  0.430667739673288

### Heuristic and Interactability
The most complex part of this portion was designing a curve for the heuristic that would make sense; we needed to ensure that a case that had no bikes or spaces would not be considered, and that an increasing quantity of something would begin to lose value compared to the increase of others. For example, it is not necessarily a better case that there will be 15 bikes waiting for you, if there is only 1 space waiting for you. We decided to go with a logarithmic curve, which when supplemented with raising the input value to a power and scaling the curve meant the difference between 1 and 5 bikes was much larger than 5-10 and etcetera. $\newline$
The next step was to develop the interactive component of the machine. To do this, we prompt the user for the necessary materials we need to produce a query for our model, then run the heuristic on the next 30 minutes from the current moment to give an optimal time to leave. The 30 minute period was hard coded, however in the future we could simply add another question that allows the user to pick length of time, or even start moment to search for. From here, we move upwards from the start point, the point of this being that if there is a tie in the heuristic, the preference should go to the earlier time. 

In [13]:
import sys
from BikeCountPredictor import BikeCountPredictor
from datetime import time as t
from datetime import datetime
import math


class BikeCountController:
    def __init__(self):
        self.oracle = BikeCountPredictor()


    def heuristic(self, startStationId, endStationId, minutes, time, temp,day, month, day_of_week, model = 'decision tree'):
        bikes = self.oracle.predictBikeCount(startStationId, minutes, temp, day, month, day_of_week, model, False)
        spaces = 26 - self.oracle.predictBikeCount(endStationId, (minutes + time), temp, day, month, day_of_week, model, False)
        if bikes == 0 or spaces == 0:
            return -sys.maxsize
        else:
            return self.curve(spaces) + self.curve(bikes)

    # Mathematical model for the heuristic to follow
    def curve(self, num):
        return 10 * math.log(num**3 + 1)


    def initiateBikeQuery(self):
        # Query user for settings
        startStation = int(input("Please enter the start station: "))
        endStation = int(input("Please enter the end station: "))
        time = int(input("Please enter the length of time for the trip in minutes: "))
        temp = int(input("What is the temperature at the moment in fahrenheit? "))
        now = datetime.now()
        minutes = (60 * now.hour) + now.minute
        day = now.day
        month = now.month
        day_of_week = now.weekday()
        leave = (0, minutes)

        for i in range(30):     # Evaluate the next 30 minutes
            heur = self.heuristic(startStation, endStation, minutes, time, temp, day, month, day_of_week)
            if (heur > leave[0]):
                leave = (heur, minutes)
            minutes = minutes + 1
        # Set result to a readable format
        minute = minutes % 60
        hour = int((minutes - minute) / 60)
        print("\nThe best moment to leave would be at", t(hour, minute, 0).strftime("%H:%M"))
        
def main():
    # Prompts for the query
    BCC = BikeCountController()
    BCC.initiateBikeQuery()


if __name__ == "__main__":
    main()


The best moment to leave would be at 21:46


In [15]:
BCC = BikeCountController()

# Sample data
# Explicit varibles, time data pulled from datetime.now()
startStationId = 342
endStationId = 12
lengthOfTrip = 15
temp = 64
BCC.initiateBikeQuery()


The best moment to leave would be at 21:48
